# Pandas基础

In [2]:
import numpy as np
import pandas as pd

In [7]:
pip install xlrd==2.0.0

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     |████████████████████████████████| 95 kB 1.6 MB/s eta 0:00:01
  Attempting uninstall: xlrd
    Found existing installation: xlrd 2.0.1
    Uninstalling xlrd-2.0.1:
      Successfully uninstalled xlrd-2.0.1
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install xlwt

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     |████████████████████████████████| 99 kB 1.6 MB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     |████████████████████████████████| 243 kB 1.9 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


## 一.文件的读取和写入
### 1.文件读取

`pandas`可以读取的文件格式有很多，这里主要介绍读取`csv, excel, txt`文件。

In [ ]:
df_csv = pd.read_csv('../data/my_csv.csv')
df_txt = pd.read_table('../data/my_table.txt')
df_excel = pd.read_excel('../data/my_excel.xlsx')

这里有一些常用的公共参数，`header=None`表示第一行不作为列名，`index_col`表示把某一列或几列作为索引，索引的内容将会在第三章进行详述，`usecols`表示读取列的集合，默认读取所有的列，`parse_dates`表示需要转化为时间的列，关于时间序列的有关内容将在第十章讲解，`nrows`表示读取的数据行数。上面这些参数在上述的三个函数里都可以使用。

In [11]:
# pd.read_table('my_table.txt', header=None)
pd.read_csv('my_csv.csv', index_col=['country'])
# pd.read_table('my_table.txt', usecols=['col1', 'col2'])
# pd.read_csv('my_csv.csv', parse_dates=['col5'])
# pd.read_excel('my_excel.xlsx', nrows=2)

,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
country,,,,,
Afghanistan,0,0,0,0.0,AS
Albania,89,132,54,4.9,EU
Algeria,25,0,14,0.7,AF
Andorra,245,138,312,12.4,EU
Angola,217,57,45,5.9,AF


在读取`txt`文件时，经常遇到分隔符非空格的情况，`read_table`有一个分割参数`sep`，它使得用户可以自定义分割符号，进行`txt`数据的读取。例如，下面的读取的表以`||||`为分割：(我的用`,`分割

In [12]:
pd.read_table('my_table.txt')

,"country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent"
0,"Afghanistan,0,0,0,0,AS"
1,"Albania,89,132,54,4.9,EU"
2,"Algeria,25,0,14,0.7,AF"
3,"Andorra,245,138,312,12.4,EU"
4,"Angola,217,57,45,5.9,AF"


上面的结果显然不是理想的，这时可以使用`sep`，同时需要指定引擎为`python`：

In [13]:
pd.read_table('my_table.txt', sep = ',', engine = 'python')

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
0,Afghanistan,0,0,0,0.0,AS
1,Albania,89,132,54,4.9,EU
2,Algeria,25,0,14,0.7,AF
3,Andorra,245,138,312,12.4,EU
4,Angola,217,57,45,5.9,AF


#### 【WARNING】`sep`是正则参数

在使用`read_table`的时候需要注意，参数`sep`中使用的是正则表达式，因此需要对`|`进行转义变成`\|`，否则无法读取到正确的结果。有关正则表达式的基本内容可以参考第八章或者其他相关资料。

#### 【END】

### 2. 数据写入

一般在数据写入中，最常用的操作是把`index`设置为`False`，特别当索引没有特殊意义的时候，这样的行为能把索引在保存的时候去除。

In [ ]:
df_csv.to_csv('../data/my_csv_saved.csv', index=False)
df_excel.to_excel('../data/my_excel_saved.xlsx', index=False)

`pandas`中没有定义`to_table`函数，但是`to_csv`可以保存为`txt`文件，并且允许自定义分隔符，常用制表符`\t`分割：

In [ ]:
df_txt.to_csv('../data/my_txt_saved.txt', sep='\t', index=False)

如果想要把表格快速转换为`markdown`和`latex`语言，可以使用`to_markdown`和`to_latex`函数，此处需要安装`tabulate`包。

In [ ]:
print(df_csv.to_markdown())
print(df_csv.to_latex())

## 二、基本数据结构

`pandas`中具有两种基本的数据结构，存储一维`values`的`Series`和二维的`DataFrame`，这两种结构上定义了很多的属性和方法

### 1.Series

`Series`一般由四个部分组成，分别是序列的值`data`、索引`index`、存储类型`dtype`、序列的名字`name`。其中，索引也可以指定它的名字，默认为空。

In [16]:
s = pd.Series(data = [100, 'a', {'dict1':5}],
             index = pd.Index(['id1', 20, 'third'], name = 'my_idx'),
             dtype = 'object',
             name  = 'my_name')
print(s)

my_idx
id1               100
20                  a
third    {'dict1': 5}
Name: my_name, dtype: object


### 【Note】`object`类型

`object`代表了一种混合类型，正如上面的例子中存储了整数、字符串以及python的字典数据结构。比外，目前`pandas`把出字符串序列也默认认为是一种`object`类型的序列，但它也可以用`string`类型存储

对于这些属性，可以通过`.`的方式来获取

利用`.shape`来获取序列的长度

In [19]:
print(s.values, s.index, s.shape)

[100 'a' {'dict1': 5}] Index(['id1', 20, 'third'], dtype='object', name='my_idx') (3,)


索引是`pandas`中最重要的概念之一，它将在第三章中被详细地讨论。如果想要取出单个索引对应的值，可以通过`[index_item]`可以取出。

### 2.DataFrame
`DataFrame`在`Series`的基础上增加了列索引，一个数据框可以由二维的`data`与列索引来创造

In [1]:
import pandas as pd
data = [[1, 'a', 1.2], [2, 'b', 2.2], [3, 'c', 3.2]]
df = pd.DataFrame(data = data,
                 index = ['row_%d'%i for i in range(3)],
                 columns = ['col_0', 'col_1', 'col_2'])
print(df)

       col_0 col_1  col_2
row_0      1     a    1.2
row_1      2     b    2.2
row_2      3     c    3.2


***但一般而言，更多的时候会采用从列索引名到数据的映射来构造数据框，同时再加上行索引***

这样可以保持每一列数据类型的一致性

In [3]:
df = pd.DataFrame(data = {'col_1' : [1, 2, 3],
                          'col_2' : list('abc'),
                          'col_3' : [1.2, 2.3, 4.5]},
                  index = ['row_%d'% i for i in range(3)])
print(df)

       col_1 col_2  col_3
row_0      1     a    1.2
row_1      2     b    2.3
row_2      3     c    4.5


由于这种映射关系，在`DataFrame`中可以用`[col_name]`与`[col_list]`来取出相应的列与由多个列组成的表，结果分别为`Series`和`DataFrame`：

In [7]:
print(df['col_1'])
print(df[['col_1', 'col_2']])

row_0    1
row_1    2
row_2    3
Name: col_1, dtype: int64
       col_1 col_2
row_0      1     a
row_1      2     b
row_2      3     c


与`Series`类似，在数据框中同样可以去除相应的属性

In [10]:
print(df.values, df.index, df.columns, df.dtypes, df.shape, sep = '\n')

[[1 'a' 1.2]
 [2 'b' 2.3]
 [3 'c' 4.5]]
Index(['row_0', 'row_1', 'row_2'], dtype='object')
Index(['col_1', 'col_2', 'col_3'], dtype='object')
col_1      int64
col_2     object
col_3    float64
dtype: object
(3, 3)


通过`.T`可以把`DataFrame`进行转置：

In [11]:
print(df.T)

      row_0 row_1 row_2
col_1     1     2     3
col_2     a     b     c
col_3   1.2   2.3   4.5


## 三、常用基本函数

In [20]:
df = pd.read_csv('drinks.csv')
df.columns

Index(['country', 'beer_servings', 'spirit_servings', 'wine_servings',
       'total_litres_of_pure_alcohol', 'continent'],
      dtype='object')

### 1.汇总函数
`head, tail`函数分别表示返回表或者序列的前`n`行和后`n`行，其中`n`默认为5：

In [14]:
print(df.head(2))
print(df.tail(3))

       country  beer_servings  spirit_servings  wine_servings  \
0  Afghanistan              0                0              0   
1      Albania             89              132             54   

   total_litres_of_pure_alcohol continent  
0                           0.0        AS  
1                           4.9        EU  
   country  beer_servings  spirit_servings  wine_servings  \
2  Algeria             25                0             14   
3  Andorra            245              138            312   
4   Angola            217               57             45   

   total_litres_of_pure_alcohol continent  
2                           0.7        AF  
3                          12.4        EU  
4                           5.9        AF  


`info, describe`分别返回表的信息概况和表中数值列对应的主要统计量 ：

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   country                       193 non-null    object 
 1   beer_servings                 193 non-null    int64  
 2   spirit_servings               193 non-null    int64  
 3   wine_servings                 193 non-null    int64  
 4   total_litres_of_pure_alcohol  193 non-null    float64
 5   continent                     170 non-null    object 
dtypes: float64(1), int64(3), object(2)
memory usage: 9.2+ KB


In [16]:
df.describe()

,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol
count,5.000000,5.000000,5.000000,5.000000
mean,115.200000,65.400000,85.000000,4.780000
std,111.023421,67.696381,128.798292,4.970614
min,0.000000,0.000000,0.000000,0.000000
25%,25.000000,0.000000,14.000000,0.700000
50%,89.000000,57.000000,45.000000,4.900000
75%,217.000000,132.000000,54.000000,5.900000
max,245.000000,138.000000,312.000000,12.400000


#### 【NOTE】更全面的数据汇总

`info, describe`只能实现较少信息的展示，如果想要对一份数据集进行全面且有效的观察，特别是在列较多的情况下，推荐使用[pandas-profiling](https://pandas-profiling.github.io/pandas-profiling/docs/)包，它将在第十一章被再次提到。

#### 【END】

### 2. 特征统计函数
在`Series`和`DataFrame`上定义了许多统计函数，最常见的是`sum, mean, median, var, std, max, min`。例如，选出身高和体重列进行演示：

此外，需要介绍的是`quantile, count, idxmax`这三个函数，它们分别返回的是分位数、非缺失值个数、最大值对应的索引：

In [22]:
df_demo = df[['beer_servings', 'spirit_servings']]
print(df_demo.quantile(0.75), df_demo.count(), df_demo.idxmax(), sep = '\n\n')

beer_servings      188.0
spirit_servings    128.0
Name: 0.75, dtype: float64

beer_servings      193
spirit_servings    193
dtype: int64

beer_servings      117
spirit_servings     68
dtype: int64


上面这些所有的函数，由于操作后返回的是标量，所以又称为聚合函数，它们有一个公共参数`axis`，默认为0代表逐列聚合，如果设置为1则表示逐行聚合：

In [25]:
df_demo.mean(axis = 1).head()

0      0.0
1    110.5
2     12.5
3    191.5
4    137.0
dtype: float64

In [3]:
import pandas as pd
df = pd.read_csv('data/learn_pandas.csv')

### 3.唯一值函数

对序列使用`unique`和`nunique`可以分别得到其唯一值组成的列表和唯一值的个数

In [8]:
df.head(6)

,School,Grade,Name,Gender,Height,Weight,Transfer,Test_Number,Test_Date,Time_Record
0,Shanghai Jiao Tong University,Freshman,Gaopeng Yang,Female,158.9,46.0,N,1,2019/10/5,0:04:34
1,Peking University,Freshman,Changqiang You,Male,166.5,70.0,N,1,2019/9/4,0:04:20
2,Shanghai Jiao Tong University,Senior,Mei Sun,Male,188.9,89.0,N,2,2019/9/12,0:05:22
3,Fudan University,Sophomore,Xiaojuan Sun,Female,NaN,41.0,N,2,2020/1/3,0:04:08
4,Fudan University,Sophomore,Gaojuan You,Male,174.0,74.0,N,2,2019/11/6,0:05:22
5,Tsinghua University,Freshman,Xiaoli Qian,Female,158.0,51.0,N,1,2019/10/31,0:03:47


In [4]:
df['School'].unique()

array(['Shanghai Jiao Tong University', 'Peking University',
       'Fudan University', 'Tsinghua University'], dtype=object)

In [5]:
df['School'].nunique()

4

得到唯一值和其对应出现的频率

In [6]:
df['School'].value_counts()

Tsinghua University              69
Shanghai Jiao Tong University    57
Fudan University                 40
Peking University                34
Name: School, dtype: int64

如果想要观察多个列组合的唯一值，可以使用`drop_duplicates`。其中关键的参数是`keep`，默认值`first`表示每个组合保留第一次出现的所在行，`last`表示保留最后一次出现的所在行，`False`表示把所有重复组合所在的行剔除

In [4]:
df_demo = df[['Gender', 'Transfer', 'Name']]
df_demo.drop_duplicates(['Gender', 'Transfer'])

,Gender,Transfer,Name
0,Female,N,Gaopeng Yang
1,Male,N,Changqiang You
12,Female,NaN,Peng You
21,Male,NaN,Xiaopeng Shen
36,Male,Y,Xiaojuan Qin
43,Female,Y,Gaoli Feng


In [5]:
df_demo.drop_duplicates(['Gender', 'Transfer'], keep = 'last')

,Gender,Transfer,Name
147,Male,NaN,Juan You
150,Male,Y,Chengpeng You
169,Female,Y,Chengquan Qin
194,Female,NaN,Yanmei Qian
197,Female,N,Chengqiang Chu
199,Male,N,Chunpeng Lv


In [7]:
df_demo.drop_duplicates(['Name', 'Gender'], keep = False).head()
# 保留只出现过一次的性别和姓名组合

,Gender,Transfer,Name
0,Female,N,Gaopeng Yang
1,Male,N,Changqiang You
2,Male,N,Mei Sun
4,Male,N,Gaojuan You
5,Female,N,Xiaoli Qian


In [10]:
#在Series中也可以使用
df['School'].drop_duplicates()

Series([], Name: School, dtype: object)

`duplcates`和`drop_duplicates`的功能类似，但前者返回了是否为唯一值的布尔列表，其`keep`参数与后者一致。其返回的序列，把重复元素设为`True`，否则为`False`。`drop_duplicates`等价于把`duplicated`为`True`的对应行剔除。

In [12]:
df_demo.duplicated(['Gender', 'Transfer']).head()

0    False
1    False
2     True
3     True
4     True
dtype: bool

In [14]:
df['School'].duplicated().head()

0    False
1    False
2     True
3    False
4     True
Name: School, dtype: bool

### 4.替换函数

一般而言，替换操作是针对某一列进行的，因此下面的例子都以`Series`举例，替换分为三类：映射替换、逻辑替换、数值替换。其中映射替换包含`replace`方法，第八章中的`str.replace`方法以及第九章中的`cat.codes`方法，此处介绍`replace`的用法。

在`replace`中，可以通过字典构造，或者传入两个列表来进行替换：

In [15]:
df['Gender'].replace({'Female':0, 'Male':1}).head()

0    0
1    1
2    1
3    0
4    1
Name: Gender, dtype: int64

In [16]:
df['Gender'].replace(['Female', 'Male'], [0, 1]).head()

0    0
1    1
2    1
3    0
4    1
Name: Gender, dtype: int64

另外`replace`还有一种特殊的方向替换，指定`method`参数为`ffill`则为用前面一个***最未被替换的值进行替换***，`bfill`则使用后面最近的未被替换的值进行替换。从下面的例子可以看到，它们的结果是不同的

In [18]:
s = pd.Series(['a', 1, 2, 'b', 2, 3, 4, 1])
s.replace([1, 2], method = 'ffill')

0    a
1    a
2    a
3    b
4    b
5    3
6    4
7    4
dtype: object

In [19]:
s.replace([1, 2], method = 'bfill')

0    a
1    b
2    b
3    b
4    3
5    3
6    4
7    1
dtype: object

#### 【WARNING】正则替换请使用`str.replace`

虽然对于`replace`而言可以使用正则替换，但是当前版本下对于`string`类型的正则替换还存在`bug`，因此如有此需求，请选择`str.replace`进行替换操作，具体的方式将在第八章中讲解。

#### 【END】

逻辑替换包括了`where`和`mask`，这两个函数是完全对称的：`where`函数在传入条件为`False`的对应行进行替换，而`mask`在传入条件为`True`的对应行进行替换，当不指定替换值时，替换为缺失值。

In [20]:
s = pd.Series([-1, 23, 10, -2])
s.where(s < 0)

0   -1.0
1    NaN
2    NaN
3   -2.0
dtype: float64

In [21]:
s.where(s<0, 100)

0     -1
1    100
2    100
3     -2
dtype: int64

In [22]:
s.mask(s<0, -10)

0   -10
1    23
2    10
3   -10
dtype: int64

需要注意的是，传入的条件只需要是与被调用的`Series`索引一直的布尔序列即可

In [23]:
condition = pd.Series([True, False, False, True], index = s.index)
s.mask(condition, -10)

0   -10
1    23
2    10
3   -10
dtype: int64

数值替换包含了`round,abs,clip`方法，它们分别按照给定的精度四舍五入，取绝对值和截断

In [27]:
s = pd.Series([-1, 1.23, 200, -50])
print(s.round(), s.abs(), s.clip(0, 2), sep = '\n\n')

0     -1.0
1      1.0
2    200.0
3    -50.0
dtype: float64

0      1.00
1      1.23
2    200.00
3     50.00
dtype: float64

0    0.00
1    1.23
2    2.00
3    0.00
dtype: float64


在clip中，超过边界的只能截断为边界值，如果要把超出边界的替换为自定义的值，应该如何做

In [46]:
s = pd.Series([-1, 1, 200, -50])
print(s.mask(s<0, -5).mask(s>20, 30))

0    -5
1     1
2    30
3    -5
dtype: int64


### 5. 排序函数
排序共有两种方式，其一为值排序，其二为索引排序，对应的函数是`sort_values`和`sort_index`。

为了演示排序函数，下面先利用`set_index`方法把年级和姓名两列作为索引，多级索引的内容和索引设置的方法将在第三章进行详细讲解。

In [49]:
df_demo = df[['Grade', 'Name', 'Height', 'Weight']].set_index(['Grade', 'Name'])
df_demo.head()

Height  Weight
Grade     Name                          
Freshman  Gaopeng Yang     158.9    46.0
          Changqiang You   166.5    70.0
Senior    Mei Sun          188.9    89.0
Sophomore Xiaojuan Sun       NaN    41.0
          Gaojuan You      174.0    74.0

对身高进行排序，默认参数`ascending = True`为升序

In [50]:
df_demo.sort_values('Height').head()

,,Height,Weight
Grade,Name,,
Junior,Xiaoli Chu,145.4,34.0
Senior,Gaomei Lv,147.3,34.0
Sophomore,Peng Han,147.8,34.0
Senior,Changli Lv,148.7,41.0
Sophomore,Changjuan You,150.5,40.0


In [51]:
df_demo.sort_values('Height', ascending = False).head()

Height  Weight
Grade    Name                         
Senior   Xiaoqiang Qin   193.9    79.0
         Mei Sun         188.9    89.0
         Gaoli Zhao      186.5    83.0
Freshman Qiang Han       185.3    87.0
Senior   Qiang Zheng     183.9    87.0

在排序中，经常遇到多列排序问题，比如在体重相同的情况下，对身高进行排序，并且保持身高降序排序，体重升序排序

In [52]:
df_demo.sort_values(['Weight', 'Height'], ascending = [True, False]).head()

,,Height,Weight
Grade,Name,,
Sophomore,Peng Han,147.8,34.0
Senior,Gaomei Lv,147.3,34.0
Junior,Xiaoli Chu,145.4,34.0
Sophomore,Qiang Zhou,150.5,36.0
Freshman,Yanqiang Xu,152.4,38.0


索引排序的用法和值排序完全一致，只不过元素的值在索引中，此时需要指定索引层的名字或者层号，用参数`level`表示，另外，需要注意的是字符串的排序顺序由字母顺序决定

In [53]:
df_demo.sort_index(level = ['Grade', 'Name'], ascending = [True, False]).head()

Height  Weight
Grade    Name                         
Freshman Yanquan Wang    163.5    55.0
         Yanqiang Xu     152.4    38.0
         Yanqiang Feng   162.3    51.0
         Yanpeng Lv        NaN    65.0
         Yanli Zhang     165.1    52.0

### 6.apply方法

`apply`方法常用于`DataFrame`的行迭代或者列迭代，他的`axis`含义与前面一致，`apply`的参数往往是一个以序列为输入的函数，例如对于`mean()`

In [55]:
df_demo = df[['Height', 'Weight']]
df_demo.apply(lambda x : x.mean())

Height    163.218033
Weight     55.015873
dtype: float64

若指定`axis=1`那么每次传入函数的就是行元素组成的`Series`，其结果与之前的逐行均值结果一致

In [57]:
df_demo.apply(lambda x : x.mean(), axis = 1).head()

0    102.45
1    118.25
2    138.95
3     41.00
4    124.00
dtype: float64

再举一个例子：`mad`函数返回的是一个序列中偏离该序列均值的绝对值大小的均值，例如序列1,3,7,10中，均值为5.25，每一个元素偏离的绝对值为4.25,2.25,1.75,4.75，这个偏离序列的均值为3.25。现在利用`apply`计算升高和体重的`mad`指标：

In [59]:
df_demo.apply(lambda x : (x - x.mean()).abs().mean())

Height     6.707229
Weight    10.391870
dtype: float64

In [60]:
df_demo.mad()

Height     6.707229
Weight    10.391870
dtype: float64

#### 【WARINING】谨慎只用`apply`

得益于传入自定义函数的处理，`apply`的自由度很高，但这是以性能为代价的。一般而言，使用`pandas`的内置函数处理和`apply`来处理同一个任务，其速度会相差较多，因此只有在确实存在自定义需求的情境下才考虑使用`apply`。

## 四、窗口对象

`pandas`中有3类窗口，分别是滑动窗口`rolling`、扩张窗口`expanding`以及指数加权窗口`ewm`。需要说明的是，以日期偏置为窗口大小的滑动窗口将在第十章讨论，指数加权窗口见本章练习。

### 1.滑动窗口

In [61]:
s = pd.Series([1, 2, 3, 4, 5])
roller = s.rolling(window = 3)
roller

Rolling [window=3,center=False,axis=0]

在得到了滑窗对象后，能够使用相应的聚合函数进行计算，需要注意的是***窗口包含当前行所在的元素***，例如在第四个位置进行均值运算时，应当计算(2 + 3 + 4)/3,而不是(1 + 2 + 3)/3

In [62]:
roller.mean()

0    NaN
1    NaN
2    2.0
3    3.0
4    4.0
dtype: float64

In [63]:
roller.sum()

0     NaN
1     NaN
2     6.0
3     9.0
4    12.0
dtype: float64

对于滑动相关系数或滑动协方差的计算，可以如下写出：

In [79]:
s2 = pd.Series([1,2,6,16,30])
roller.cov(s2)

0     NaN
1     NaN
2     2.5
3     7.0
4    12.0
dtype: float64

In [80]:
roller.corr(s2)

0         NaN
1         NaN
2    0.944911
3    0.970725
4    0.995402
dtype: float64

此外，还支持使用`apply`传入自定义函数，其传入值是对应窗口的`Series`，例如上述的均值函数可以等效表示：

In [81]:
roller.apply(lambda x:x.mean())

0    NaN
1    NaN
2    2.0
3    3.0
4    4.0
dtype: float64

`shift, diff, pct_change`是一组滑窗函数，它们的公共参数为`periods = n`，默认为1，
- 分别表示取向前第`n`个元素的值
- 向前第`n`个元素做差（与`numpy`不同，后者表示`n`阶差分）
- 与向前第`n`个元素相比计算增长率

这里`n`可以为负，表示反方向的类似操作

In [64]:
s = pd.Series([1, 2, 3, 4, 5, 6])
s.shift(3)

0    NaN
1    NaN
2    NaN
3    1.0
4    2.0
5    3.0
dtype: float64

In [65]:
s.diff(3)

0    NaN
1    NaN
2    NaN
3    3.0
4    3.0
5    3.0
dtype: float64

In [79]:
s.pct_change() # 后除以前-1
def my_pct(x):
    l = list(x)
    return l[-1] / l[0] - 1

s.rolling(2).apply(my_pct)

0         NaN
1    1.000000
2    0.500000
3    0.333333
4    0.250000
5    0.200000
dtype: float64

#### 【练一练】

`rolling`对象的默认窗口方向都是向前的，某些情况下用户需要向后的窗口，例如对1,2,3设定向后窗口为2的`sum`操作，结果为3,5,NaN，此时应该如何实现向后的滑窗操作？

可以将Series先反过来再进行计算

In [80]:
s = pd.Series([1, 2, 3])
s[::-1].rolling(2).sum()[::-1]

0    3.0
1    5.0
2    NaN
dtype: float64

### 2.扩张窗口

扩张窗口又称累计窗口，可以理解为一个动态长度的窗口，其窗口大小就是从序列开始处到具体操作的对应位置，其使用的聚合函数会作用于这些逐步扩张的窗口上。具体地说，设序列为a1, a2, a3, a4，则其每个位置对应的窗口即\[a1\]、\[a1, a2\]、\[a1, a2, a3\]、\[a1, a2, a3, a4\]。

In [81]:
s = pd.Series([1, 3, 4, 10])
s.expanding().mean()

0    1.000000
1    2.000000
2    2.666667
3    4.500000
dtype: float64

#### 【练一练】

`cummax, cumsum, cumprod`函数是典型的类扩张窗口函数，请使用`expanding`对象依次实现它们。

In [82]:
s.expanding().sum()

0     1.0
1     4.0
2     8.0
3    18.0
dtype: float64

In [85]:
s.expanding().max()

0     1.0
1     3.0
2     4.0
3    10.0
dtype: float64

## 五、练习
### Ex1：口袋妖怪数据集
现有一份口袋妖怪的数据集，下面进行一些背景说明：

* `#`代表全国图鉴编号，不同行存在相同数字则表示为该妖怪的不同状态

* 妖怪具有单属性和双属性两种，对于单属性的妖怪，`Type 2`为缺失值
* `Total, HP, Attack, Defense, Sp. Atk, Sp. Def, Speed`分别代表种族值、体力、物攻、防御、特攻、特防、速度，其中种族值为后6项之和

In [1]:
import pandas as pd
df = pd.read_csv('data/Pokemon.csv')
df.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65


1. 对`HP, Attack, Defense, Sp. Atk, Sp. Def, Speed`进行加总，验证是否为`Total`值。

2. 对于`#`重复的妖怪只保留第一条记录，解决以下问题：

* 求第一属性的种类数量和前三多数量对应的种类
* 求第一属性和第二属性的组合种类
* 求尚未出现过的属性组合

3. 按照下述要求，构造`Series`：

* 取出物攻，超过120的替换为`high`，不足50的替换为`low`，否则设为`mid`
* 取出第一属性，分别用`replace`和`apply`替换所有字母为大写
* 求每个妖怪六项能力的离差，即所有能力中偏离中位数最大的值，添加到`df`并从大到小排序

In [6]:
(df[df.columns[5:]].sum(1) == df['Total']).all()

True

In [ ]:
dp_dup = df.drop_duplicates('#', keep = 'first')
dp_dup['Type 1'].nunique() # 18
dp_dup['Type 1'].value_counts().index[:3]
# Index(['Water', 'Normal', 'Grass'], dtype='object')

In [12]:
attr_dup = dp_dup.drop_duplicates(['Type 1', 'Type 2'])
attr_dup.shape[0]

143

In [12]:
L_full = [i + ' ' + j if i != j else i for i in df['Type 1'].unique() for j in df['Type 1'].unique()]
L_part = [i + ' ' + j if not isinstance(j, float) else i for i, j in zip(df['Type 1'], df['Type 2'])]
# 因为part要求是表中有的，所以要用zip绑定，并且j可能为nan是浮点，不能和str加起来
res = set(L_full).difference(set(L_part))
len(res)

170

In [8]:
df['Attack'].mask(df['Attack']>120, 'high').mask(df['Attack']<50, 'low').mask((50<=df['Attack'])&(df['Attack']<=120), 'mid')

0       low
1       mid
2       mid
3       mid
4       mid
       ... 
795     mid
796    high
797     mid
798    high
799     mid
Name: Attack, Length: 800, dtype: object

In [12]:
df['Type 1'].replace({i:str.upper(i) for i in df['Type 1'].unique()})
df['Type 1'].apply(lambda x : str.upper(x)).head()

0    GRASS
1    GRASS
2    GRASS
3    GRASS
4     FIRE
Name: Type 1, dtype: object

In [15]:
df['Deviation'] = df[df.columns[5:]].apply(lambda x : max((x - x.median()).abs()), axis = 1)
df.sort_values('Deviation', ascending = False).head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Deviation
230,213,Shuckle,Bug,Rock,505,20,10,230,10,230,5,215.0
121,113,Chansey,Normal,NaN,450,250,5,5,35,105,50,207.5
261,242,Blissey,Normal,NaN,540,255,10,10,75,135,55,190.0
333,306,AggronMega Aggron,Steel,NaN,630,70,140,230,60,80,50,155.0
224,208,SteelixMega Steelix,Steel,Ground,610,75,125,230,55,95,30,145.0


In [16]:
df['Deviation'].tail()

795    50.0
796    60.0
797    55.0
798    65.0
799    30.0
Name: Deviation, dtype: float64

### Ex2：指数加权窗口
1. 作为扩张窗口的`ewm`窗口

在扩张窗口中，用户可以使用各类函数进行历史的累计指标统计，但这些内置的统计函数往往把窗口中的所有元素赋予了同样的权重。事实上，可以给出不同的权重来赋给窗口中的元素，指数加权窗口就是这样一种特殊的扩张窗口。

其中，最重要的参数是`alpha`，它决定了默认情况下的窗口权重为$w_i=(1−\alpha)^i,i\in\{0,1,...,t\}$，其中$i=t$表示当前元素，$i=0$表示序列的第一个元素。

从权重公式可以看出，离开当前值越远则权重越小，若记原序列为$x$，更新后的当前元素为$y_t$，此时通过加权公式归一化后可知：

$$
\begin{split}y_t &=\frac{\sum_{i=0}^{t} w_i x_{t-i}}{\sum_{i=0}^{t} w_i} \\
&=\frac{x_t + (1 - \alpha)x_{t-1} + (1 - \alpha)^2 x_{t-2} + ...
+ (1 - \alpha)^{t} x_{0}}{1 + (1 - \alpha) + (1 - \alpha)^2 + ...
+ (1 - \alpha)^{t}}\\\end{split}
$$

对于`Series`而言，可以用`ewm`对象如下计算指数平滑后的序列：

In [1]:
import numpy as np
import pandas as pd
np.random.seed(100)
s = pd.Series(np.random.randint(-1, 2, 30).cumsum())
s.head()

0   -1
1   -2
2   -3
3   -2
4   -1
dtype: int64

In [2]:
s.ewm(alpha = 0.2).mean().head()

0   -1.000000
1   -1.555556
2   -2.147541
3   -2.097561
4   -1.771061
dtype: float64

用`expanding`窗口实现

In [13]:
def ewm_func(x, alpha = 0.2):
    win = (1 - alpha) ** np.arange(x.shape[0])[::-1]
    res = (win * x).sum() / win.sum()
    return res
s.expanding().apply(ewm_func).head()

0   -1.000000
1   -1.555556
2   -2.147541
3   -2.097561
4   -1.771061
dtype: float64

2.作为滑动窗口的`ewm`窗口

从第1问中可以看到，`ewm`作为一种扩张窗口的特例，只能从序列的第一个元素开始加权。现在希望给定一个限制窗口`n`，只对包含自身的最近的`n`个元素作为窗口进行滑动加权平滑。请根据滑窗函数，给出新的`wi`与`yt`的更新公式，并通过`rolling`窗口实现这一功能。

#### 2.

新的权重为$w_i = (1 - \alpha)^i, i\in \{0,1,...,n-1\}$，$y_t$更新如下：
$$
\begin{split}y_t &=\frac{\sum_{i=0}^{n-1} w_i x_{t-i}}{\sum_{i=0}^{n-1} w_i} \\
&=\frac{x_t + (1 - \alpha)x_{t-1} + (1 - \alpha)^2 x_{t-2} + ...
+ (1 - \alpha)^{n-1} x_{t-(n-1)}}{1 + (1 - \alpha) + (1 - \alpha)^2 + ...
+ (1 - \alpha)^{n-1}}\\\end{split}
$$


In [10]:
s.rolling(4).apply(ewm_func).head()

0         NaN
1         NaN
2         NaN
3   -2.097561
4   -1.878049
dtype: float64